# Building Python packages

A carefully crafted directory structure makes the extension to building packages from the code relatively easy. One has to be careful since applying 'build' and 'setuptools' requires giving those tools some TLC; after all this time, they are still a little clunky. This notebook will show an example.

Such a structure is in the 'buildable' directory, following the structure from https://packaging.python.org/en/latest/tutorials/packaging-projects/

In [11]:
!tree -I __pycache__ ../buildable

../buildable
└── number_returns
    ├── number_returns_tests
    │   ├── __init__.py
    │   ├── ints
    │   │   ├── __init__.py
    │   │   └── test_gimmes.py
    │   └── strs
    │       ├── __init__.py
    │       └── test_gimme_strs.py
    └── src
        └── number_returns
            ├── __init__.py
            ├── ints
            │   ├── gimmes.py
            │   └── __init__.py
            └── strs
                ├── gimme_strs.py
                └── __init__.py

8 directories, 10 files


In [12]:
!cd ../buildable; python -m pytest

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/buildable
collected 3 items                                                              

number_returns/number_returns_tests/ints/test_gimmes.py ..               [ 66%]
number_returns/number_returns_tests/strs/test_gimme_strs.py .            [100%]

============================== 3 passed in 0.02s ===============================


The package tree structures are labelled with \_\_init\_\_.py files in the directories of the hierarchy, and _only_ in the directories of the hierarchy. This encapsulates the folder inclusions to the "tests" and 2nd "number_returns" directories, and allows any self-import structure within number_returns to work straight from the code, or from the installed package in exactly the same way. Indeed, the code under the second "number_returns" is exactly what will be installed into the site-packages directory on a pip install.

The only hack that needs to be put in place is a sys.path mod in tests/\_\_init\_\_.py (which is at least entirely scoped to that set of tests), which bridges the two trees described by the \_\_init\_\_.py files between "tests" and "number_returns". 

The trees for the installable code and tests are deliberately separate since only one of the trees will be packaged into a pip install-able module. The tests module is given its own name (rather than just being called tests) so that "buildable" can have multiple modules, each with their own test directory, and pytest will not get confused by multiple modules called "tests" (this confusion can be removed by treating the multiple test modules as namespace packages, but this then runs the risks of test name clashes in the multiple tests folders).

In [13]:
!cat ../buildable/number_returns/number_returns_tests/__init__.py

import os
import sys

sys.path.append(os.path.join(os.path.dirname(__file__), "..", "src"))


Whilst strictly speaking not necessary, the \_\_init\_\_.py files enforce that the package builds will be "regular packages" as opposed to "namespace packages". The default action for developers should be making a "regular package", as this enforces tighter scoping.

https://stackoverflow.com/questions/37139786/is-init-py-not-required-for-packages-in-python-3-3/

https://packaging.python.org/en/latest/guides/packaging-namespace-packages/